Group Name: Suicide Squad

Calling the packages we will use for the project, along with the plotly username and api key

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from scipy.stats import norm, skew
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
import seaborn as sns
import json
import requests
import warnings
from requests.auth import HTTPBasicAuth

def ignore_warn(*args, **kwargs):
    pass

warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

username = 'moonkang' # Replace with YOUR USERNAME
api_key = 'nKkigC26m95bqBAK52af' # Replace with YOUR API KEY

auth = HTTPBasicAuth(username, api_key)
headers = {'Plotly-Client-Platform': 'python'}

plotly.tools.set_credentials_file(username=username, api_key=api_key)

pd.set_option('display.max_columns', None)

This area is for deleting plots in plotly (run this block if the # of plots is already maxed out)

In [2]:
#for deleting plots in plotly
def get_pages(username, page_size):
    url = 'https://api.plot.ly/v2/folders/all?user='+username+'&page_size='+str(page_size)
    response = requests.get(url, auth=auth, headers=headers)
    if response.status_code != 200:
        return
    page = json.loads(response.content)
    yield page
    while True:
        resource = page['children']['next'] 
        if not resource:
            break
        response = requests.get(resource, auth=auth, headers=headers)
        if response.status_code != 200:
            break
        page = json.loads(response.content)
        yield page
        
def permanently_delete_files(username, page_size=500, filetype_to_delete='plot'):
    for page in get_pages(username, page_size):
        for x in range(0, len(page['children']['results'])):
            fid = page['children']['results'][x]['fid']
            res = requests.get('https://api.plot.ly/v2/files/' + fid, auth=auth, headers=headers)
            res.raise_for_status()
            if res.status_code == 200:
                json_res = json.loads(res.content)
                if json_res['filetype'] == filetype_to_delete:
                    # move to trash
                    requests.post('https://api.plot.ly/v2/files/'+fid+'/trash', auth=auth, headers=headers)
                    # permanently delete
                    requests.delete('https://api.plot.ly/v2/files/'+fid+'/permanent_delete', auth=auth, headers=headers)

permanently_delete_files(username, filetype_to_delete='plot')
permanently_delete_files(username, filetype_to_delete='grid')

# Loading the data

Load the data from .csv and looking at the head of the data and dimensions

In [33]:
## load training data
train = pd.read_csv("train.csv", header = 0, index_col=None)
## load test data
test = pd.read_csv("test.csv", header = 0, index_col=None)

print(train.shape)
print(test.shape)
train.head()

(1460, 81)
(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Exploratory Data Analysis

In [4]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Based on [Serigne's](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard) kernel, we are dropping the ID column.

In [34]:
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

Added the sum of Basement and Livable Area as a column for the scatterplot matrix

In [35]:
train['LivBsmtArea']=train['TotalBsmtSF'] + train['GrLivArea']
test['LivBsmtArea']=test['TotalBsmtSF'] + test['GrLivArea'] #also applied it to the test set

Building the Scatterplot Matrix, just for us to see the relationship of the target variable with variables we think will have a big impact to the price

In [7]:
dataframe = train[['SalePrice', 'GrLivArea', 'TotalBsmtSF', 'LivBsmtArea']].copy()

fig = ff.create_scatterplotmatrix(dataframe, height=1000, width=1000, diag='histogram', size=3, title="House Train Variables")
py.iplot(fig)

Remove the outliers and log transform the sale price

In [36]:
#Outlier removal
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

#Log transform
train["LogSalePrice"] = np.log1p(train["SalePrice"])

Plot the data without the sales outliers

In [9]:
dataframe = train[['LogSalePrice', 'GrLivArea', 'TotalBsmtSF', 'LivBsmtArea']].copy()

fig = ff.create_scatterplotmatrix(dataframe, height=1000, width=1000, diag='histogram', size=3, title="House Train Variables")
py.iplot(fig)

In [10]:
SalePrice = go.Histogram(x=train['SalePrice'], cumulative=dict(enabled=True))
LogSalePrice = go.Histogram(x=train['LogSalePrice'], cumulative=dict(enabled=True))
fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(SalePrice, 1, 1)
fig.append_trace(LogSalePrice, 1, 2)

fig['layout'].update(height=400, width=800)

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [11]:
corrmat = train.corr()
y = list(corrmat.columns.values)
x = list(corrmat.index.values)
z = [list(corrmat[x]) for x in corrmat]

In [12]:
trace = go.Heatmap(z=z, x=x, y=y, colorscale='Viridis')
data=[trace]
py.iplot(data)

# Data Cleaning process

Combining Train and Test Data sets. Ideally, we should apply the same process on a separate place to avoid data leakage, but for this project, we combined them and did the cleaning process to make our jupyter notebook shorter.

In [37]:
ntrain = train.shape[0]
ntest = test.shape[0]
saleprice_train = train.SalePrice.values
logsaleprice_train = train.LogSalePrice.values
all_df = pd.concat((train, test)).reset_index(drop=True)
all_df.drop(['SalePrice'], axis=1, inplace=True)
all_df.drop(['LogSalePrice'], axis=1, inplace=True)
print("Dimensions - {}".format(all_df.shape))

Dimensions - (2917, 80)


Counting the missingness on each column

In [38]:
all_df_na = (all_df.isnull().sum() / len(all_df)) * 100
all_df_na = all_df_na.drop(all_df_na[all_df_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_df_na})
missing_data.head(10)

data = [go.Bar(x=all_df_na.index, y=all_df_na, marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5
        )))]

layout = go.Layout(
    xaxis=dict(tickangle=-90), title="% of Missingness"
)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [39]:
#Handling features guided by data description

all_df.loc[:, "Alley"] = all_df.loc[:, "Alley"].fillna("None")
all_df.loc[:, "BsmtQual"] = all_df.loc[:, "BsmtQual"].fillna("NoBasmt")
all_df.loc[:, "BsmtCond"] = all_df.loc[:, "BsmtCond"].fillna("NoBasmt")
all_df.loc[:, "BsmtExposure"] = all_df.loc[:, "BsmtExposure"].fillna("NoBasmt")
all_df.loc[:, "BsmtFinType1"] = all_df.loc[:, "BsmtFinType1"].fillna("NoBasmt")
all_df.loc[:, "BsmtFinType2"] = all_df.loc[:, "BsmtFinType2"].fillna("NoBasmt")
all_df.loc[:, "BsmtFullBath"] = all_df.loc[:, "BsmtFullBath"].fillna(0)
all_df.loc[:, "BsmtHalfBath"] = all_df.loc[:, "BsmtHalfBath"].fillna(0)
all_df.loc[:, "BsmtUnfSF"] = all_df.loc[:, "BsmtUnfSF"].fillna(0)
all_df.loc[:, "Fence"] = all_df.loc[:, "Fence"].fillna("NoFnc")
all_df.loc[:, "FireplaceQu"] = all_df.loc[:, "FireplaceQu"].fillna("NoFrplc")
all_df.loc[:, "Fireplaces"] = all_df.loc[:, "Fireplaces"].fillna(0)
all_df.loc[:, "Functional"] = all_df.loc[:, "Functional"].fillna("Typ")
all_df.loc[:, "GarageType"] = all_df.loc[:, "GarageType"].fillna("NoGrg")
all_df.loc[:, "GarageFinish"] = all_df.loc[:, "GarageFinish"].fillna("NoGrg")
all_df.loc[:, "GarageQual"] = all_df.loc[:, "GarageQual"].fillna("NoGrg")
all_df.loc[:, "GarageCond"] = all_df.loc[:, "GarageCond"].fillna("NoGrg")
all_df.loc[:, "GarageArea"] = all_df.loc[:, "GarageArea"].fillna(0)
all_df.loc[:, "GarageCars"] = all_df.loc[:, "GarageCars"].fillna(0)
all_df.loc[:, "MiscFeature"] = all_df.loc[:, "MiscFeature"].fillna("NoMscFtr")
all_df.loc[:, "MiscVal"] = all_df.loc[:, "MiscVal"].fillna(0)
all_df.loc[:, "PoolQC"] = all_df.loc[:, "PoolQC"].fillna("NoPool")
all_df.loc[:, "PoolArea"] = all_df.loc[:, "PoolArea"].fillna(0)


In [40]:
#check ot frontage check vs lot area
trace = go.Scatter(
    x = all_df['LotFrontage'],
    y = all_df['LotArea'],
    mode = 'markers'
)

layout = go.Layout(
    xaxis=dict(
        title='Lot Frontage'),
    yaxis=dict(
        title='Lot Area')
)

data = [trace]

fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

In [41]:
index = all_df['LotFrontage'].index[all_df['LotFrontage'].apply(np.isnan)]
LFna_df = all_df.loc[index, :]
LFna_df.head(10)

#conclusion: the NaN in LotFrontage are zero because the column does not differentiate zero form NaN
#the column does not have zero as minimum based on describe
#NaN in lotfrontage = zero (fill with 0)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LivBsmtArea,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
7,1107,983,0,None,3,1Fam,TA,Mn,859.0,32.0,ALQ,BLQ,1.0,0.0,Gd,216.0,Y,PosN,Norm,SBrkr,228,TA,TA,HdBoard,HdBoard,NoFnc,TA,2,CBlock,2,Typ,484.0,2.0,TA,RFn,TA,Attchd,1973.0,2090,1,GasA,Ex,2Story,1,TA,Lvl,Gtl,3197.0,10382,Corner,NaN,IR1,0,60,RL,240.0,Stone,Shed,350,11,NWAmes,204,6,7,Y,0,NoPool,CompShg,Gable,Normal,WD,0,Pave,7,1107.0,AllPub,235,1973,1973,2009
12,912,0,0,None,2,1Fam,TA,No,737.0,0.0,ALQ,Unf,1.0,0.0,TA,175.0,Y,Norm,Norm,SBrkr,0,TA,TA,HdBoard,Plywood,NoFnc,NoFrplc,0,CBlock,1,Typ,352.0,1.0,TA,Unf,TA,Detchd,1962.0,912,0,GasA,TA,1Story,1,TA,Lvl,Gtl,1824.0,12968,Inside,NaN,IR2,0,20,RL,0.0,None,NoMscFtr,0,9,Sawyer,0,6,5,Y,0,NoPool,CompShg,Hip,Normal,WD,176,Pave,4,912.0,AllPub,140,1962,1962,2008
14,1253,0,0,None,2,1Fam,TA,No,733.0,0.0,BLQ,Unf,1.0,0.0,TA,520.0,Y,Norm,Norm,SBrkr,176,TA,TA,MetalSd,MetalSd,GdWo,Fa,1,CBlock,1,Typ,352.0,1.0,TA,RFn,TA,Attchd,1960.0,1253,1,GasA,TA,1Story,1,TA,Lvl,Gtl,2506.0,10920,Corner,NaN,IR1,0,20,RL,212.0,BrkFace,NoMscFtr,0,5,NAmes,213,5,6,Y,0,NoPool,CompShg,Hip,Normal,WD,0,Pave,5,1253.0,AllPub,0,1960,1960,2008
16,1004,0,0,None,2,1Fam,TA,No,578.0,0.0,ALQ,Unf,1.0,0.0,TA,426.0,Y,Norm,Norm,SBrkr,0,TA,TA,Wd Sdng,Wd Sdng,NoFnc,TA,1,CBlock,1,Typ,480.0,2.0,TA,Fin,TA,Attchd,1970.0,1004,0,GasA,Ex,1Story,1,TA,Lvl,Gtl,2008.0,11241,CulDSac,NaN,IR1,0,20,RL,180.0,BrkFace,Shed,700,3,NAmes,0,7,6,Y,0,NoPool,CompShg,Gable,Normal,WD,0,Pave,5,1004.0,AllPub,0,1970,1970,2010
24,1060,0,0,None,3,1Fam,TA,Mn,188.0,668.0,Rec,ALQ,1.0,0.0,TA,204.0,Y,Norm,Norm,SBrkr,0,Gd,TA,Plywood,Plywood,MnPrv,TA,1,CBlock,1,Typ,270.0,1.0,TA,Unf,TA,Attchd,1968.0,1060,0,GasA,Ex,1Story,1,Gd,Lvl,Gtl,2120.0,8246,Inside,NaN,IR1,0,20,RL,0.0,None,NoMscFtr,0,5,Sawyer,90,8,5,Y,0,NoPool,CompShg,Gable,Normal,WD,0,Pave,6,1060.0,AllPub,406,1968,2001,2010
31,1228,0,0,None,3,1Fam,TA,No,0.0,0.0,Unf,Unf,0.0,0.0,TA,1228.0,Y,Norm,Norm,SBrkr,0,TA,TA,HdBoard,HdBoard,MnPrv,NoFrplc,0,CBlock,1,Typ,271.0,1.0,TA,Unf,TA,Attchd,1966.0,1228,1,GasA,Gd,1Story,1,Gd,Lvl,Gtl,2456.0,8544,CulDSac,NaN,IR1,0,20,RL,0.0,None,NoMscFtr,0,6,Sawyer,65,6,5,Y,0,NoPool,CompShg,Gable,Normal,WD,0,Pave,6,1228.0,AllPub,0,1966,2006,2008
42,884,0,0,None,2,1Fam,TA,Av,747.0,93.0,ALQ,LwQ,1.0,0.0,Gd,0.0,Y,Norm,Norm,SBrkr,0,TA,TA,HdBoard,HdBoard,MnPrv,NoFrplc,0,CBlock,1,Typ,504.0,2.0,Gd,RFn,TA,Attchd,1983.0,884,0,GasA,Gd,SFoyer,1,Gd,Lvl,Gtl,1724.0,9180,CulDSac,NaN,IR1,0,85,RL,0.0,None,NoMscFtr,0,12,SawyerW,0,7,5,Y,0,NoPool,CompShg,Gable,Normal,WD,0,Pave,5,840.0,AllPub,240,1983,1983,2007
43,938,0,0,None,3,1Fam,TA,Av,280.0,491.0,LwQ,BLQ,1.0,0.0,Gd,167.0,Y,Norm,Norm,SBrkr,0,TA,TA,VinylSd,VinylSd,MnPrv,NoFrplc,0,CBlock,1,Typ,308.0,1.0,TA,Unf,TA,Detchd,1977.0,938,0,GasA,TA,1Story,1,TA,Lvl,Gtl,1876.0,9200,CulDSac,NaN,IR1,0,20,RL,0.0,None,NoMscFtr,0,7,CollgCr,0,6,5,Y,0,NoPool,CompShg,Hip,Normal,WD,0,Pave,5,938.0,AllPub,145,1975,1980,2008
50,794,676,0,None,3,1Fam,TA,Av,182.0,0.0,GLQ,Unf,0.0,1.0,Gd,612.0,Y,Norm,Norm,SBrkr,0,TA,TA,VinylSd,VinylSd,NoFnc,NoFrplc,0,PConc,2,Typ,388.0,2.0,TA,Fin,TA,Attchd,1997.0,1470,0,GasA,Gd,2Story,1,TA,Lvl,Gtl,2264.0,13869,Corner,NaN,IR2,0,60,RL,0.0,None,NoMscFtr,0,7,Gilbert,75,6,6,Y,0,NoPool,CompShg,Gable,Normal,WD,0,Pave,6,794.0,AllPub,0

In [42]:
# By missing at random, decided to fill NA with 0 for BedroomAbvGr as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "BedroomAbvGr"] = all_df.loc[:, "BedroomAbvGr"].fillna(0)

# By missing at random, decided to fill NA with N for CentralAir as missingness can be interpreted to be directly 
# related to lack of Central Air
all_df.loc[:, "CentralAir"] = all_df.loc[:, "CentralAir"].fillna("N")

# By missing at random, decided to fill NA with Norm for Condition1 & Condition2 as missingness can be interpreted to be directly 
# related to lack of proximity to any conditions
all_df.loc[:, "Condition1"] = all_df.loc[:, "Condition1"].fillna("Norm")
all_df.loc[:, "Condition2"] = all_df.loc[:, "Condition2"].fillna("Norm")

# By missing at random, decided to fill NA with 0 for EnclosedPorch as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "EnclosedPorch"] = all_df.loc[:, "EnclosedPorch"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for ExterCond & ExterQual
# since we assume it will have minumum impact
all_df.loc[:, "ExterCond"] = all_df.loc[:, "ExterCond"].fillna("TA")
all_df.loc[:, "ExterQual"] = all_df.loc[:, "ExterQual"].fillna("TA")

# By missing at random, decided to fill NA with 0 for HalfBath as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "HalfBath"] = all_df.loc[:, "HalfBath"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for HeatingQC 
# since we assume it will have minumum impact
all_df.loc[:, "HeatingQC"] = all_df.loc[:, "HeatingQC"].fillna("TA")

# By missing at random, decided to fill NA with 0 for KitchenAbvGr as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "KitchenAbvGr"] = all_df.loc[:, "KitchenAbvGr"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for KitchenQual 
# since we assume it will have minumum impact
all_df.loc[:, "KitchenQual"] = all_df.loc[:, "KitchenQual"].fillna("TA")

# *****LotFrontage : NA most likely means no lot frontage - missing at random*****
all_df.loc[:, "LotFrontage"] = all_df.loc[:, "LotFrontage"].fillna(0)

# By missing completely at random, decided to fill NA with Reg (averaging) for LotShape 
# since we assume it will have minumum impact
all_df.loc[:, "LotShape"] = all_df.loc[:, "LotShape"].fillna("Reg")

# By missing at random, decided to fill NA with None and 0 for MasVnrType & MasVnrArea as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "MasVnrType"] = all_df.loc[:, "MasVnrType"].fillna("None")
all_df.loc[:, "MasVnrArea"] = all_df.loc[:, "MasVnrArea"].fillna(0)

# By missing at random, decided to fill NA with 0 for OpenPorchSF as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "OpenPorchSF"] = all_df.loc[:, "OpenPorchSF"].fillna(0)

# By missing at random, decided to fill NA with N for PavedDrive as missingness can be interpreted to be directly 
# related to No Paved Drive
all_df.loc[:, "PavedDrive"] = all_df.loc[:, "PavedDrive"].fillna("N")

# By missing completely at random, decided to fill NA with Normal (averaging) for SaleCondition 
# since we assume it will have minumum impact
all_df.loc[:, "SaleCondition"] = all_df.loc[:, "SaleCondition"].fillna("Normal")

# By missing at random, decided to fill NA with 0 for ScreenPorch as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "ScreenPorch"] = all_df.loc[:, "ScreenPorch"].fillna(0)

# By missing at random, decided to fill NA with 0 for TotRmsAbvGrd as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "TotRmsAbvGrd"] = all_df.loc[:, "TotRmsAbvGrd"].fillna(0)

# By missing at random, decided to fill NA with AllPub for Utilities as missingness can be interpreted to be directly 
# related to All public Utilities 
all_df.loc[:, "Utilities"] = all_df.loc[:, "Utilities"].fillna("AllPub")

# By missing at random, decided to fill NA with 0 for WoodDeckSF as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "WoodDeckSF"] = all_df.loc[:, "WoodDeckSF"].fillna(0)

In [43]:
# There are variables with numerical vaues that after investigating in the Description we find that are actually
# best described as categories
# Changing the month sold and Year sold variables into categorical is a great example of 
# using dummy variables to adjust for seasonality

all_df = all_df.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

all_df['YrSold'] = [str(x) for x in all_df['YrSold']]

In [44]:
# Encoding categorical features as ordered numbers after gathering inside of order from description

all_df = all_df.replace({"Alley" : {"None": 0, "Grvl" : 1, "Pave" : 2},
                       "BsmtCond" : {"NoBasmt" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"NoBasmt" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"NoBasmt" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"NoBasmt" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"NoBasmt" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "FireplaceQu" : {"NoFrplc" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"NoGrg" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"NoGrg" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"NoPool" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}}
                     )

In [45]:
#THIS IS TO BE DISCUSSED LATER, LEAVE COMMENTED OUT




# Create new features
# 1* Simplifications of existing features
all_df["SimplOverallQual"] = all_df.OverallQual.replace({1 : 1, 2 : 1, 3 : 1, # bad
                                                       4 : 2, 5 : 2, 6 : 2, # average
                                                       7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                      })
all_df["SimplOverallCond"] = all_df.OverallCond.replace({1 : 1, 2 : 1, 3 : 1, # bad
                                                       4 : 2, 5 : 2, 6 : 2, # average
                                                       7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                      })
all_df["SimplPoolQC"] = all_df.PoolQC.replace({1 : 1, 2 : 1, # average
                                             3 : 2, 4 : 2 # good
                                            })
all_df["SimplGarageCond"] = all_df.GarageCond.replace({1 : 1, # bad
                                                     2 : 1, 3 : 1, # average
                                                     4 : 2, 5 : 2 # good
                                                    })
all_df["SimplGarageQual"] = all_df.GarageQual.replace({1 : 1, # bad
                                                     2 : 1, 3 : 1, # average
                                                     4 : 2, 5 : 2 # good
                                                    })
all_df["SimplFireplaceQu"] = all_df.FireplaceQu.replace({1 : 1, # bad
                                                       2 : 1, 3 : 1, # average
                                                       4 : 2, 5 : 2 # good
                                                      })
all_df["SimplFireplaceQu"] = all_df.FireplaceQu.replace({1 : 1, # bad
                                                       2 : 1, 3 : 1, # average
                                                       4 : 2, 5 : 2 # good
                                                      })
all_df["SimplFunctional"] = all_df.Functional.replace({1 : 1, 2 : 1, # bad
                                                     3 : 2, 4 : 2, # major
                                                     5 : 3, 6 : 3, 7 : 3, # minor
                                                     8 : 4 # typical
                                                    })
all_df["SimplKitchenQual"] = all_df.KitchenQual.replace({1 : 1, # bad
                                                       2 : 1, 3 : 1, # average
                                                       4 : 2, 5 : 2 # good
                                                      })
all_df["SimplHeatingQC"] = all_df.HeatingQC.replace({1 : 1, # bad
                                                   2 : 1, 3 : 1, # average
                                                   4 : 2, 5 : 2 # good
                                                  })
all_df["SimplBsmtFinType1"] = all_df.BsmtFinType1.replace({1 : 1, # unfinished
                                                         2 : 1, 3 : 1, # rec room
                                                         4 : 2, 5 : 2, 6 : 2 # living quarters
                                                        })
all_df["SimplBsmtFinType2"] = all_df.BsmtFinType2.replace({1 : 1, # unfinished
                                                         2 : 1, 3 : 1, # rec room
                                                         4 : 2, 5 : 2, 6 : 2 # living quarters
                                                        })
all_df["SimplBsmtCond"] = all_df.BsmtCond.replace({1 : 1, # bad
                                                 2 : 1, 3 : 1, # average
                                                 4 : 2, 5 : 2 # good
                                                })
all_df["SimplBsmtQual"] = all_df.BsmtQual.replace({1 : 1, # bad
                                                 2 : 1, 3 : 1, # average
                                                 4 : 2, 5 : 2 # good
                                                })
all_df["SimplExterCond"] = all_df.ExterCond.replace({1 : 1, # bad
                                                   2 : 1, 3 : 1, # average
                                                   4 : 2, 5 : 2 # good
                                                  })
all_df["SimplExterQual"] = all_df.ExterQual.replace({1 : 1, # bad
                                                   2 : 1, 3 : 1, # average
                                                   4 : 2, 5 : 2 # good
                                                  })

# 2* Combinations of existing features
# Overall quality of the house
all_df["OverallGrade"] = all_df["OverallQual"] * all_df["OverallCond"]
# Overall quality of the garage
all_df["GarageGrade"] = all_df["GarageQual"] * all_df["GarageCond"]
# Overall quality of the exterior
all_df["ExterGrade"] = all_df["ExterQual"] * all_df["ExterCond"]
# Overall kitchen score
all_df["KitchenScore"] = all_df["KitchenAbvGr"] * all_df["KitchenQual"]
# Overall fireplace score
all_df["FireplaceScore"] = all_df["Fireplaces"] * all_df["FireplaceQu"]
# Overall garage score
all_df["GarageScore"] = all_df["GarageArea"] * all_df["GarageQual"]
# Overall pool score
all_df["PoolScore"] = all_df["PoolArea"] * all_df["PoolQC"]
# Simplified overall quality of the house
all_df["SimplOverallGrade"] = all_df["SimplOverallQual"] * all_df["SimplOverallCond"]
# Simplified overall quality of the exterior
all_df["SimplExterGrade"] = all_df["SimplExterQual"] * all_df["SimplExterCond"]
# Simplified overall pool score
all_df["SimplPoolScore"] = all_df["PoolArea"] * all_df["SimplPoolQC"]
# Simplified overall garage score
all_df["SimplGarageScore"] = all_df["GarageArea"] * all_df["SimplGarageQual"]
# Simplified overall fireplace score
all_df["SimplFireplaceScore"] = all_df["Fireplaces"] * all_df["SimplFireplaceQu"]
# Simplified overall kitchen score
all_df["SimplKitchenScore"] = all_df["KitchenAbvGr"] * all_df["SimplKitchenQual"]
# Total number of bathrooms
all_df["TotalBath"] = all_df["BsmtFullBath"] + (0.5 * all_df["BsmtHalfBath"]) + \
all_df["FullBath"] + (0.5 * all_df["HalfBath"])




##### DO NOT Uncomment this. I already made this column in the scatterplot above #####
# # Total SF for house (incl. basement)
# train["AllSF"] = train["GrLivArea"] + train["TotalBsmtSF"]


# Total SF for 1st + 2nd floors
all_df["AllFlrsSF"] = all_df["1stFlrSF"] + all_df["2ndFlrSF"]
# Total SF for porch
all_df["AllPorchSF"] = all_df["OpenPorchSF"] + all_df["EnclosedPorch"] + \
all_df["3SsnPorch"] + all_df["ScreenPorch"]
# Has masonry veneer or not
all_df["HasMasVnr"] = all_df.MasVnrType.replace({"BrkCmn" : 1, "BrkFace" : 1, "CBlock" : 1, 
                                               "Stone" : 1, "None" : 0})
# House completed before sale or not
all_df["BoughtOffPlan"] = all_df.SaleCondition.replace({"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, 
                                                      "Family" : 0, "Normal" : 0, "Partial" : 1})

Box Cox Transformation for skewed features

In [46]:
numeric_feats = all_df.dtypes[all_df.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_df[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

,Skew
PoolScore,22.357445
SimplPoolScore,22.332942
MiscVal,21.939672
PoolQC,20.341424
SimplPoolQC,18.985317
PoolArea,17.688664
LotArea,13.109495
LowQualFinSF,12.084539
3SsnPorch,11.372080
KitchenAbvGr,4.300550


In [47]:
skewness = skewness[abs(skewness) > 0.75]
print("{} features to Box Cox Transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    all_df[feat] = boxcox1p(all_df[feat], lam)

86 features to Box Cox Transform


In [48]:
all_df = pd.get_dummies(all_df)
print(all_df.shape)

(2917, 296)


# Re-assigning to Test and Train

In [49]:
train = all_df[:ntrain]
test = all_df[ntrain:]